# Code Generator

The requirement: use an Open Source model to generate high performance C++ code from Python code

To replicate this, you'll need to set up a HuggingFace endpoint as I do in the video. It's simple to do, and it's quite satisfying to see the results!

It's also an important part of your learning; this is the first example of deploying an open source model to be behind an API. We'll return to this in Week 8, but this should plant a seed in your mind for what's involved in moving open source models into production.

In [40]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
import unittest

In [2]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [3]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [4]:
system_message = "You are an assistant that helps to create unit tests for python code. "
system_message += "Respond only with unit tests for the given python code that will exersize all test cases."
system_message += "Make nessecary comments and document as needed to help deveopers undersand what the unit tests are doing and the use cases that are being tested."

In [5]:
def user_prompt_for(python):
    user_prompt = "Create Unit test for the following python code that will cover all test cases."
    user_prompt += "Respond only with Python code. "
    user_prompt += python
    return user_prompt

In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [11]:
# write to a file called optimized.cpp

def write_output(cpp, filename):
    code = cpp.replace("```cpp","").replace("```","")
    with open(f"{filename}.py", "w") as f:
        f.write(code)

In [17]:
def create_unit_tests_gpt(python, save_file_name):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply, save_file_name)

In [9]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [18]:
create_unit_tests_gpt(pi, 'test_calculate_pi')

```python
import unittest

# Import the function to be tested
from your_module import calculate  # Assuming the main code is in 'your_module.py'

class TestCalculateFunction(unittest.TestCase):
    
    def test_zero_iterations(self):
        # Test case when iterations are zero
        self.assertEqual(calculate(0, 4, 1), 1.0)

    def test_single_iteration(self):
        # Test single iteration
        result = calculate(1, 4, 1)
        self.assertAlmostEqual(result, 1 + (1/3) - (1/5), places=12)

    def test_multiple_iterations(self):
        # Test some custom value for the iterations
        result = calculate(2, 4, 1)
        # The expected result is calculated manually or using a reliable method
        expected_result = 1 + (1/3) - (1/5) + (1/7) - (1/9)
        self.assertAlmostEqual(result, expected_result, places=12)

    def test_negative_param1(self):
        # Test case when param1 is negative
        result = calculate(1, -4, 1)
        self.assertAlmostEqual(result, ca

In [14]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [19]:
create_unit_tests_gpt(python_hard, 'test_total_max_subarray')

```python
import unittest

class TestLcgAndSubarraySum(unittest.TestCase):
    def test_lcg(self):
        # Test the LCG generator with a known seed
        gen = lcg(12345)
        self.assertEqual(next(gen), 87628868)
        self.assertEqual(next(gen), 71072467)
        self.assertEqual(next(gen), 2332836379)
        self.assertEqual(next(gen), 2540403940)

    def test_max_subarray_sum_small_range(self):
        # Test with a small range of random numbers
        n = 5
        seed = 1
        min_val = -2
        max_val = 2
        # Generating the array expected from (min_val, max_val)
        # The possible range is limited and you might manually calculate expected result
        self.assertEqual(max_subarray_sum(n, seed, min_val, max_val), 4) # this should be calculated based on LCG

    def test_max_subarray_sum_large_random_numbers(self):
        # Test max subarray sum with a large n to ensure handling of larger sizes
        n = 10000
        seed = 42
        min_val = -

In [33]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [36]:
def create_unit_test_stream(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far        

In [37]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="Unit Test:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Create Unit Test code")

    convert.click(create_unit_test_stream, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [28]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [29]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [39]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Test Python Code")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        unitTest = gr.Textbox(label="Unit Test code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        create_test = gr.Button("Create Unit tests code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        unit_test_run = gr.Button("Run Python Unit tests")
        # cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        unit_test_out = gr.TextArea(label="Unit Test result:", elem_classes=["unitTest"])

    create_test.click(create_unit_test_stream, inputs=[python, model], outputs=[unitTest])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    unit_test_run.click(execute_python, inputs=[unitTest], outputs=[unit_test_out])
    # cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

Traceback (most recent call last):
  File "D:\Users\joeco\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Users\joeco\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Users\joeco\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Users\joeco\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1731, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Users\joeco\anaconda3\envs\llms\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return awai

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Thank you to @CloudLlama for an amazing contribution</h2>
            <span style="color:#090;">
                A student has contributed a chunk of code to improve this, in the next 2 cells. You can now select which Python porgram to run,
                and a compiler is automatically selected that will work on PC, Windows and Mac. Massive thank you @CloudLlama!
            </span>
        </td>
    </tr>
</table>